# Experimentation to calibrate model

This notebook contains notes and results of several runs in order to determine the best approach to match ingredients in the taxonomy. The idea is to maximise the similarity score on a given sample, starting from a baseline -that is- the most simple model.

In [1]:
### Import packages

import os
import re
import json
import time
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from wordcloud import WordCloud
import matplotlib.pyplot as plt

from sklearn.metrics.pairwise import cosine_similarity

from sentence_transformers import SentenceTransformer
sentence_model = SentenceTransformer('paraphrase-mpnet-base-v2')

import warnings
warnings.filterwarnings('ignore')

In [2]:
### Define constants

RANDOM_SEED = 888
SAMPLE_SIZE = 5000

CSV_PATH = 'calibration/csv/'
EXCEL_PATH = 'calibration/excel/'
JSON_PATH = 'json/'

# Filenames
FILENAME_RAW_TAXONOMY = 'raw_taxonomy.csv'
FILENAME_RAW_INGREDIENTS = 'raw_ingredients.csv'
FILENAME_RAW_INGREDIENTS_SAMPLE = 'raw_ingredients_sample.csv'
FILENAME_STOPWORDS_EXCEL = 'raw_stopwords.xlsx'
FILENAME_STOPWORDS_JSON = 'stopwords.json'

In [3]:
### Define functions

def file_exists(filename, directory = ''):
    
    pwd = os.getcwd()
    path = os.path.join(pwd, directory, filename)
    
    return os.path.isfile(path);
                          
def load_df_from_csv(filename, directory = ''):
    
    pwd = os.getcwd()
    path = os.path.join(pwd, directory, filename)
    
    with open(path, 'r') as f:
        df = pd.read_csv(f)
        print(f'Table successfuly loaded from {filename}')
        
    return df;

def generate_string_connection():
    
    # Retrieve params from env
    DB_HOST = os.environ['DB_HOST_STAGE']
    DB_NAME = os.environ['DB_NAME_STAGE']
    DB_PORT = os.environ['DB_PORT_STAGE']
    DB_USERNAME = os.environ['DB_USERNAME_STAGE']
    DB_PASSWORD = os.environ['DB_PASSWORD_STAGE']
    
    # Create string
    string_conn = f'mysql+pymysql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
    
    return string_conn;

def load_df_from_sql(tableName, string_conn):
    
    sqlEngine = create_engine(string_conn)
    dbConnection = sqlEngine.connect()
    
    try:
        df = pd.read_sql_table(tableName, dbConnection)
        print(f'Query successfully executed for table {tableName}')
        return df;

    except Exception as ex:   
        print(ex)

    finally:
        dbConnection.close();
        
def vectorise_sentence(sentence_list, MODEL = sentence_model):
    
    INIT_TIME = time.time()
    
    dict_return = {}
    array_vectors = MODEL.encode(sentence_list)
    
    dict_return['sentence'] = sentence_list
    dict_return['vector'] = array_vectors
    
    FINAL_TIME = time.time() - INIT_TIME
    print(f'Vectorisation finished. Total time: {FINAL_TIME:.2f} seconds')
    
    return dict_return;

def calculate_similarity(dict_taxonomy, dict_ingredients):

    # Calculate similarity
    array_similarity = cosine_similarity(dict_ingredients['vector'], dict_taxonomy['vector'])

    # Get best match
    best_score_idx = np.argmax(array_similarity, axis = 1)

    # Assign ingredient and score to df
    matched_ingredient = [dict_taxonomy['sentence'][i] for i in best_score_idx]
    matched_score = [array_similarity[i][j] for i, j in enumerate(best_score_idx)]

    dict_return = {'ingredient': dict_ingredients['sentence'],
                  'matched_ingredient': matched_ingredient,
                  'matched_score': matched_score}
    
    return dict_return;

## From this point, the experiment begins

In [4]:
### Import files to workspace from directory. If file does not exist, from database

STRING_CONNECTION = generate_string_connection()

# Import taxonomy
if file_exists(FILENAME_RAW_TAXONOMY, CSV_PATH):
    df_raw_taxonomy = load_df_from_csv(FILENAME_RAW_TAXONOMY, CSV_PATH)
    
else:
    df_raw_taxonomy = load_df_from_sql('kp_ingredient_taxonomy', STRING_CONNECTION)
    
    df_raw_taxonomy.to_csv(os.path.join(CSV_PATH, FILENAME_RAW_TAXONOMY), index = False)
    
# Import raw ingredients
if file_exists(FILENAME_RAW_INGREDIENTS, CSV_PATH):
    df_raw_ingredients = load_df_from_csv(FILENAME_RAW_INGREDIENTS, CSV_PATH)
    
else:
    df_raw_ingredients = load_df_from_sql('kp_ingredients_base_pantry', STRING_CONNECTION)
    
    df_raw_ingredients.to_csv(os.path.join(CSV_PATH, FILENAME_RAW_INGREDIENTS), index = False)

Table successfuly loaded from raw_taxonomy.csv
Table successfuly loaded from raw_ingredients.csv


In [5]:
### Load sample dataset. If non existant, generate from raw file

if file_exists(FILENAME_RAW_INGREDIENTS_SAMPLE, CSV_PATH):
    df_sample_ingredients = load_df_from_csv(FILENAME_RAW_INGREDIENTS_SAMPLE, CSV_PATH)
    
else:
    df_sample_ingredients = df_raw_ingredients.sample(SAMPLE_SIZE, random_state = RANDOM_SEED)
    
    df_sample_ingredients.to_csv(os.path.join(CSV_PATH, FILENAME_RAW_INGREDIENTS_SAMPLE), index = False)

Table successfuly loaded from raw_ingredients_sample.csv


In [6]:
### Round 1: vectorise per category and compute scores

INIT_TIME = time.time()

list_categories = df_raw_taxonomy['category_id'].unique()

dict_taxonomy_categorised = {}
dict_ingredients_categorised = {}
dict_matched_ingredients = {}

for CAT in list_categories:
    
    list_taxonomy_categorised = df_raw_taxonomy[df_raw_taxonomy['category_id'] == CAT]['ingredient_name'].tolist()
    list_ingredient_categorised = df_sample_ingredients[df_sample_ingredients['category_id'] == CAT]['ingredient_name'].tolist()
    
    if len(list_taxonomy_categorised) == 0:
        print(f'Did not find any ingredient for category {CAT} in taxonomy. Moving on.')
        continue
        
    elif len(list_ingredient_categorised) == 0:
        print(f'Did not find any ingredient for category {CAT} in ingredients. Moving on.')
        continue
        
    dict_taxonomy = vectorise_sentence(list_taxonomy_categorised)
    dict_ingredients = vectorise_sentence(list_ingredient_categorised)
        
    dict_taxonomy_categorised[CAT] = dict_taxonomy
    dict_ingredients_categorised[CAT] = dict_ingredients
    
    dict_matched_ingredients[CAT] = calculate_similarity(dict_taxonomy, dict_ingredients)
    
    print(f'Finalised with category {CAT}. Moving on.')

FINAL_TIME = time.time() - INIT_TIME
    
print(f'Finalised all ingredients. Total time: {FINAL_TIME:.2f} seconds')

Vectorisation finished. Total time: 1.88 seconds
Vectorisation finished. Total time: 17.15 seconds
Finalised with category 101. Moving on.
Vectorisation finished. Total time: 1.23 seconds
Vectorisation finished. Total time: 55.92 seconds
Finalised with category 102. Moving on.
Vectorisation finished. Total time: 2.51 seconds
Vectorisation finished. Total time: 39.71 seconds
Finalised with category 103. Moving on.
Vectorisation finished. Total time: 0.63 seconds
Vectorisation finished. Total time: 0.24 seconds
Finalised with category 105. Moving on.
Vectorisation finished. Total time: 2.71 seconds
Vectorisation finished. Total time: 25.72 seconds
Finalised with category 107. Moving on.
Vectorisation finished. Total time: 2.20 seconds
Vectorisation finished. Total time: 9.55 seconds
Finalised with category 108. Moving on.
Vectorisation finished. Total time: 1.76 seconds
Vectorisation finished. Total time: 5.94 seconds
Finalised with category 109. Moving on.
Vectorisation finished. Total 

In [7]:
### Create dict to save results

dict_results = {'round': [],
               'description': [],
               'execution_time': [],
               'average_score': [],
               'ratio_over_80': []}

In [8]:
### Round 1: compute results

dict_categories = {key: [key] * len(content['ingredient']) for key, content in dict_matched_ingredients.items()}

df_category_results = pd.DataFrame(columns = ['category_id', 'ingredient', 'matched_ingredient', 'matched_score'])

for CAT in dict_categories.keys():
    
    DF = pd.DataFrame({'category_id': dict_categories[CAT]} | dict_matched_ingredients[CAT])
    df_category_results = pd.concat([df_category_results, DF])
    
df_category_results.reset_index(drop = True, inplace = True)
    
AVG_SCORE = df_category_results['matched_score'].mean()
RATIO_OVER_80 = sum(df_category_results['matched_score'] >= 0.8) / len(df_category_results)
    
print(f'Time to compute: {FINAL_TIME:.2f} seconds')
print(f'Average matching score: {AVG_SCORE:.3f}')
print(f'Ratio over 0.8: {RATIO_OVER_80:.3f}')

# Append results
dict_results['round'].append(1)
dict_results['description'].append('Baseline model')
dict_results['execution_time'].append(FINAL_TIME)
dict_results['average_score'].append(AVG_SCORE)
dict_results['ratio_over_80'].append(RATIO_OVER_80)

Time to compute: 194.83 seconds
Average matching score: 0.631
Ratio over 0.8: 0.120


In [9]:
### Round 2: run all possible combinations

INIT_TIME = time.time()

list_taxonomy = df_raw_taxonomy['ingredient_name'].tolist()
list_ingredient = df_sample_ingredients['ingredient_name'].tolist()

dict_taxonomy = vectorise_sentence(list_taxonomy)
dict_ingredients = vectorise_sentence(list_ingredient)

dict_similarity = calculate_similarity(dict_taxonomy, dict_ingredients)

df_all_results = pd.DataFrame(dict_similarity)
AVG_SCORE = df_all_results['matched_score'].mean()
RATIO_OVER_80 = sum(df_all_results['matched_score'] >= 0.8) / len(df_all_results)

FINAL_TIME = time.time() - INIT_TIME

print(f'Time to compute: {FINAL_TIME:.2f} seconds')
print(f'Average matching score: {AVG_SCORE:.3f}')
print(f'Ratio over 0.8: {RATIO_OVER_80:.3f}')

# Append results
dict_results['round'].append(2)
dict_results['description'].append('All combinations')
dict_results['execution_time'].append(FINAL_TIME)
dict_results['average_score'].append(AVG_SCORE)
dict_results['ratio_over_80'].append(RATIO_OVER_80)

Vectorisation finished. Total time: 15.19 seconds
Vectorisation finished. Total time: 171.40 seconds
Time to compute: 186.65 seconds
Average matching score: 0.663
Ratio over 0.8: 0.133


In [10]:
### Round 3: remove numbers

INIT_TIME = time.time()

list_taxonomy = df_raw_taxonomy['ingredient_name'].tolist()
list_ingredient = df_sample_ingredients['ingredient_name'].tolist()

list_removed_numbers_ingredients = [re.sub(r'[0-9]+', '', i) for i in list_ingredient]

dict_taxonomy = vectorise_sentence(list_taxonomy)
dict_removed_numbers_ingredients = vectorise_sentence(list_removed_numbers_ingredients)

dict_similarity = calculate_similarity(dict_taxonomy, dict_removed_numbers_ingredients)

df_noNumbers_results = pd.DataFrame(dict_similarity)
AVG_SCORE = df_noNumbers_results['matched_score'].mean()
RATIO_OVER_80 = sum(df_noNumbers_results['matched_score'] >= 0.8) / len(df_noNumbers_results)

FINAL_TIME = time.time() - INIT_TIME

print(f'Time to compute: {FINAL_TIME:.2f} seconds')
print(f'Average matching score: {AVG_SCORE:.3f}')
print(f'Ratio over 0.8: {RATIO_OVER_80:.3f}')

# Append results
dict_results['round'].append(3)
dict_results['description'].append('Numbers removed')
dict_results['execution_time'].append(FINAL_TIME)
dict_results['average_score'].append(AVG_SCORE)
dict_results['ratio_over_80'].append(RATIO_OVER_80)

Vectorisation finished. Total time: 14.92 seconds
Vectorisation finished. Total time: 153.62 seconds
Time to compute: 168.63 seconds
Average matching score: 0.673
Ratio over 0.8: 0.154


In [11]:
### Round 4: remove numbers and signs

INIT_TIME = time.time()

list_taxonomy = df_raw_taxonomy['ingredient_name'].tolist()
list_ingredient = df_sample_ingredients['ingredient_name'].tolist()

list_removed_numbers_ingredients = [re.sub(r'[^A-Za-z ]+', '', i) for i in list_ingredient]

dict_taxonomy = vectorise_sentence(list_taxonomy)
dict_removed_numbers_ingredients = vectorise_sentence(list_removed_numbers_ingredients)

dict_similarity = calculate_similarity(dict_taxonomy, dict_removed_numbers_ingredients)

df_noNumbersAndSigns_results = pd.DataFrame(dict_similarity)
AVG_SCORE = df_noNumbersAndSigns_results['matched_score'].mean()
RATIO_OVER_80 = sum(df_noNumbersAndSigns_results['matched_score'] >= 0.8) / len(df_noNumbersAndSigns_results)

FINAL_TIME = time.time() - INIT_TIME

print(f'Time to compute: {FINAL_TIME:.2f} seconds')
print(f'Average matching score: {AVG_SCORE:.3f}')
print(f'Ratio over 0.8: {RATIO_OVER_80:.3f}')

# Append results
dict_results['round'].append(4)
dict_results['description'].append('Remove all but characters and whitespace')
dict_results['execution_time'].append(FINAL_TIME)
dict_results['average_score'].append(AVG_SCORE)
dict_results['ratio_over_80'].append(RATIO_OVER_80)

Vectorisation finished. Total time: 14.88 seconds
Vectorisation finished. Total time: 118.73 seconds
Time to compute: 133.67 seconds
Average matching score: 0.671
Ratio over 0.8: 0.151


In [12]:
### Round 5: remove frequent, uninformative words

INIT_TIME = time.time()

list_taxonomy = df_raw_taxonomy['ingredient_name'].tolist()
list_ingredient = df_sample_ingredients['ingredient_name'].tolist()

# Remove numbers
list_removed_numbers_ingredients = [re.sub(r'[^A-Za-z ]+', '', i) for i in list_ingredient] # Preserve only letters and whitespace
list_removed_numbers_ingredients = [re.sub(' +', ' ', i) for i in list_removed_numbers_ingredients] # Remove double whitespace

list_ingredient_words = [i.split(' ') for i in list_removed_numbers_ingredients]
list_ingredients_flatten = [food.lower().strip() for sublist in list_ingredient_words for food in sublist]

list_taxonomy_words = [i.split('-') for i in list_taxonomy]
list_taxonomy_flatten = [food.lower() for sublist in list_taxonomy_words for food in sublist]

df_word_count_ingredients = pd.DataFrame(pd.value_counts(np.array(list_ingredients_flatten)))
df_word_count_taxonomy = pd.DataFrame(pd.value_counts(np.array(list_taxonomy_flatten)))

# Remove used words in taxonomy
df_word_count_ingredients.reset_index(inplace = True)
df_word_count_ingredients.rename(columns = {'index': 'word', 0: 'count'}, inplace = True)

df_word_count_taxonomy.reset_index(inplace = True)
df_word_count_taxonomy.rename(columns = {'index': 'word', 0: 'count'}, inplace = True)

df_word_count_ingredients = pd.merge(df_word_count_ingredients,
                                    df_word_count_taxonomy[['word']],
                                    how = 'left',
                                    on = 'word',
                                    indicator = True)

df_word_count_ingredients = df_word_count_ingredients[df_word_count_ingredients['_merge'] == 'left_only']
df_word_count_ingredients.set_index('word', inplace = True)

# Plot wordcloud
# dict_frequencies = df_word_count_ingredients['count'].to_dict()

# fig = plt.figure(1, (13, 7))
# ax = fig.add_subplot(1, 1, 1)

# wordcloud = WordCloud(max_font_size = 50,
#                       max_words = 100,
#                       background_color = 'white').generate_from_frequencies(dict_frequencies)

# ax.imshow(wordcloud, interpolation = 'bilinear')
# plt.axis('off')
# plt.show()

# Clean stopwords
list_stopwords = df_word_count_ingredients.index.tolist()
dict_stopwords = vectorise_sentence(list_stopwords)

dict_similarity_stopwords = calculate_similarity(dict_taxonomy, dict_stopwords)
df_stopwords = pd.DataFrame(dict_similarity_stopwords)

df_stopwords_out = pd.merge(df_word_count_ingredients[['count']],
                           df_stopwords,
                           how = 'left',
                           left_index = True,
                           right_on = 'ingredient')

if not file_exists(FILENAME_STOPWORDS_EXCEL, EXCEL_PATH):
    df_stopwords_out.to_excel(os.path.join(EXCEL_PATH, FILENAME_STOPWORDS_EXCEL), index = False)

Vectorisation finished. Total time: 24.48 seconds


In [13]:
# Round 5. Execution without stopwords

INIT_TIME = time.time()

# Import stopwords
if file_exists(FILENAME_STOPWORDS_JSON, JSON_PATH):
    with open(os.path.join(JSON_PATH, FILENAME_STOPWORDS_JSON), 'r') as f:
        dict_stopwords = json.load(f)

# Create lists
list_taxonomy = df_raw_taxonomy['ingredient_name'].tolist()
list_ingredient = df_sample_ingredients['ingredient_name'].tolist()

# Process ingredients list
list_removed_numbers_ingredients = [re.sub(r'[^A-Za-z ]+', ' ', i) for i in list_ingredient] # Remove all but letters and spaces
list_removed_numbers_ingredients = [re.sub(r' +', ' ', i) for i in list_removed_numbers_ingredients] # Remove double spaces
list_removed_numbers_ingredients = [i.lower() for i in list_removed_numbers_ingredients] # lower caps

# Remove stopwords
double_list_ingredients = [i.split(' ') for i in list_removed_numbers_ingredients]
double_list_ingredients_sliced = [[i for i in sublist if i not in dict_stopwords['stopwords']] for sublist in double_list_ingredients]

# Reconstruct cleaned list
list_ingredients_clean = [' '.join(i).strip() for i in double_list_ingredients_sliced]

# Encode lists
dict_taxonomy = vectorise_sentence(list_taxonomy)
dict_ingredients_clean = vectorise_sentence(list_ingredients_clean)

dict_similarity = calculate_similarity(dict_taxonomy, dict_ingredients_clean)

df_clean_results = pd.DataFrame(dict_similarity)
df_clean_results['original_ingredient'] = list_ingredient

AVG_SCORE = df_clean_results['matched_score'].mean()
RATIO_OVER_80 = sum(df_clean_results['matched_score'] >= 0.8) / len(df_clean_results)

FINAL_TIME = time.time() - INIT_TIME

print(f'Time to compute: {FINAL_TIME:.2f} seconds')
print(f'Average matching score: {AVG_SCORE:.3f}')
print(f'Ratio over 0.8: {RATIO_OVER_80:.3f}')

# Append results
dict_results['round'].append(5)
dict_results['description'].append('Remove unwanted words')
dict_results['execution_time'].append(FINAL_TIME)
dict_results['average_score'].append(AVG_SCORE)
dict_results['ratio_over_80'].append(RATIO_OVER_80)

Vectorisation finished. Total time: 12.66 seconds
Vectorisation finished. Total time: 82.70 seconds
Time to compute: 95.52 seconds
Average matching score: 0.719
Ratio over 0.8: 0.294


In [14]:
### Create dataframe with result metrics

df_results = pd.DataFrame(dict_results)
df_results

,round,description,execution_time,average_score,ratio_over_80
0,1,Baseline model,194.833881,0.630795,0.1196
1,2,All combinations,186.653683,0.662786,0.1332
2,3,Numbers removed,168.631898,0.672659,0.1544
3,4,Remove all but characters and whitespace,133.670261,0.670832,0.1506
4,5,Remove unwanted words,95.517525,0.719051,0.2936


File examples for automation

In [85]:
### Prepare vectorised ingredients file

list_taxonomy = df_raw_taxonomy['ingredient_name'].tolist()

dict_taxonomy = vectorise_sentence(list_taxonomy)

sr_taxonomy = pd.Series(data = df_raw_taxonomy['id'].tolist(), index = df_raw_taxonomy['ingredient_name'])
dict_tax_ids = sr_taxonomy.to_dict()

dict_taxonomy_vectorised = {dict_tax_ids[content]: list(dict_taxonomy['vector'][pos].astype(float)) for pos, content in enumerate(dict_taxonomy['sentence'])}

with open(f'{JSON_PATH}/taxonomy_vectorised.json', 'w') as F:
    json.dump(dict_taxonomy_vectorised, F, indent = 1)

Vectorisation finished. Total time: 13.03 seconds


In [86]:
### Samples of input and output

df_io_examples = df_clean_results.sample(10, random_state = 888)
df_io_examples = pd.merge(df_io_examples,
                         df_raw_taxonomy[['id', 'ingredient_name']],
                         how = 'left',
                         left_on = 'matched_ingredient',
                         right_on = 'ingredient_name')

# Input
dict_p1_input = {'body': df_io_examples['original_ingredient'].tolist()}

with open(f'{JSON_PATH}/P1_input.json', 'w') as F:
    json.dump(dict_p1_input, F, indent = 1)
    
# Output
dict_io_examples = df_io_examples[['original_ingredient', 'id', 'matched_score']].to_dict(orient = 'tight')
dict_p1_output = {i: dict_io_examples['data'][i] for i in dict_io_examples['index']}

with open(f'{JSON_PATH}/P1_output.json', 'w') as F:
    json.dump(dict_p1_output, F, indent = 1)